In [1]:
import pandas as pd
import fiona
import geopandas as gpd
from sqlalchemy import create_engine
import numpy as np
#import dedupe
from dotenv import load_dotenv
import os
import pyArango
import usaddress
from usaddress import tag
from scourgify import normalize_address_record
import re
import json

In [2]:
from pyArango.connection import *
conn = Connection(username="root", password="0505")

# open arangoDB database
corp_db = conn["corp_data"]

# arangoDB database structure:

# corp_db: arangoDB database
# corp_db["corp_data"]: arangoDB collection
# corp_db["corp_data"][17777999]: arangoDB document (one piece of data, or one corporation)

In [3]:
load_dotenv()
PG_CONNECT = os.getenv("PG_CONNECT")

In [6]:
# extracting the values of the corp_data_processed collection
val_aql = "FOR x IN corp_data_processed RETURN x"
value_query_result = corp_db.AQLQuery(val_aql,rawResults=True, batchSize = 1000)
col_value = {}
ind_val = 0

for value in value_query_result:
    col_value[ind_val] = value
    ind_val += 1
    
# create dataframe from dictionary of dictionaries
corp = pd.DataFrame.from_dict(data = col_value, orient = 'index')
corp_ar = corp.to_numpy()

['17777999' 'corp_data_processed/17777999' '_eZCy3XK---' '00wepr'
 'Domestic Limited Liability Company (LLC)' 'MILLER PLACE LLC'
 '189 WELLS AVE, STE 302, NEWTON CENTRE, MA 02459' 'BARRY S FOX'
 '189 WELLS AVE, STE 302, NEWTON CENTRE, MA 02459']

 0: _key
 1: _id
 2: _rev
 3: DataID
 4: EntityType
 5: EntityName
 6: EntityAddress
 7: AgentName
 8: AgentAddress

In [7]:
# extracting the values of the indiv_data_processed collection
indiv_aql = "FOR x IN indiv_unique_address RETURN x"
indiv_query_result = corp_db.AQLQuery(indiv_aql,rawResults=True, batchSize = 1000)
col_indiv = {}
ind_indiv = 0

for indiv in indiv_query_result:
    col_indiv[ind_indiv] = indiv
    ind_indiv += 1

indiv = pd.DataFrame.from_dict(data = col_indiv, orient = 'index')
indiv_ar = indiv.to_numpy()

In [8]:
def corp2indiv(corp_df, indiv_df, batch_size):
    
    num_batch = int(corp_df.shape[0] / batch_size)
    
    for i in range(0,num_batch+1):
        print(i)
        print(i*batch_size,(i+1)*batch_size)
        batch_df = corp_df[i*batch_size:(i+1)*batch_size][:]
            
        for j in range(0,batch_df.shape[0]):
            cid = batch_df[j][3]
            #print(batch_df[j])
            corp_id = batch_df[j][1]
            c_indiv = np.where(indiv_df[:,3] == cid)[0]
            #print(indiv_df[c_indiv])
            indiv_keys = indiv_df[c_indiv][:,0]
            for key in indiv_keys:
                indiv = corp_db["indiv_unique_address"].fetchDocument(key)
                indiv_edge = corp_db["indiv2indiv"].getEdges(indiv,rawResults = True)[0]
                indiv_u = indiv_edge['_from']
                edge_attributes = {"_from": corp_id, "_to": indiv_u}
                edge = corp_db["corp2indiv_final"].createDocument(edge_attributes)
                edge.save()

corp2indiv(corp_ar,indiv_ar,1000)

920
920000 921000
921
921000 922000
922
922000 923000
923
923000 924000
924
924000 925000
925
925000 926000
926
926000 927000
927
927000 928000
928
928000 929000
929
929000 930000
930
930000 931000
931
931000 932000
932
932000 933000
933
933000 934000
934
934000 935000
935
935000 936000
936
936000 937000
937
937000 938000
938
938000 939000
939
939000 940000
940
940000 941000
941
941000 942000
942
942000 943000
943
943000 944000
944
944000 945000
945
945000 946000
946
946000 947000
947
947000 948000
948
948000 949000
949
949000 950000
950
950000 951000
951
951000 952000
952
952000 953000
953
953000 954000
954
954000 955000
955
955000 956000
956
956000 957000
957
957000 958000
958
958000 959000
959
959000 960000
960
960000 961000
961
961000 962000
962
962000 963000
963
963000 964000
964
964000 965000
965
965000 966000
966
966000 967000
967
967000 968000
968
968000 969000
969
969000 970000
970
970000 971000
971
971000 972000
972
972000 973000
973
973000 974000
974
974000 975000
975
975000